## Giới thiệu vấn đề
  1. Sau khi sử dụng mô hình dự đoán chúng ta có sẽ kết hợp với dataframe để tìm những giao dịch gần giống nhau trên hệ thống. Sau đó sẽ đánh giá các giao dịch gian lận hay không ?

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/My Drive/Đồ Án Tốt Nghiệp Machine Learning/'

/content/drive/My Drive/Đồ Án Tốt Nghiệp Machine Learning


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import sys
sys.path.append('/content/drive/My Drive/LDS6_NguyenTanHuy/')
import ML_Model as md
import ML_Phantich as pt
import ML_Transform as mt

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from datetime import datetime

# Đọc dữ liệu mới nhất từ hệ thống

In [ ]:
df = pd.read_csv("Data_new.csv")

In [ ]:
df.head()

,Account,Group,Reason,Time,Type,Symbol,Volume,Price,Swap,Balance,Bonus,Reg.date,Profit,Comment
0,5346027,GF-B-JPY-GM2,Mobile,2020.11.18 02:48:44,buy,xauusd,0.4,1880.64000,0.0,229440.0,169427.0,2020.08.18 04:59,-1998.0,NaN
1,5346027,GF-B-JPY-GM2,Mobile,2020.11.18 02:43:36,buy,xauusd,0.8,1881.58000,0.0,229440.0,169427.0,2020.08.18 04:59,-11823.0,NaN
2,5354537,GF-B-JPY-GM2,Mobile,2020.11.18 02:48:43,buy limit,gbpjpy,4.0,137.85700,NaN,325539.0,405240.0,2020.09.13 13:03,NaN,NaN
3,5354537,GF-B-JPY-GM2,Mobile,2020.11.18 01:28:39,sell,eurchf,4.0,1.08099,0.0,325539.0,405240.0,2020.09.13 13:03,-914.0,NaN
4,5354537,GF-B-JPY-GM2,Mobile,2020.11.17 17:15:44,sell limit,audcad,4.0,0.96064,NaN,325539.0,405240.0,2020.09.13 13:03,NaN,NaN


In [ ]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
df.rename(columns={"Login":"Account","Credit":"Bonus"},inplace=True)

In [ ]:
df_transform = df.copy()

In [ ]:
df_transform = df_transform.merge(df_transform.groupby(by=['Account'])['Type'].count().reset_index(),how="inner",left_on=["Account"],right_on=["Account"],validate="many_to_one",copy=False)
df_transform.rename(columns={"Type_x":"Type","Type_y":"Num_of_order"},inplace=True)

In [ ]:
df_transform.Time = pd.to_datetime(df_transform.Time)
now = datetime.now()
df_transform.Time = (now - df_transform.Time).dt.total_seconds()

In [ ]:
df_transform = df_transform.merge(df_transform.groupby(by=['Account',"Type"])['Time'].median().reset_index(),how="inner",left_on=["Account","Type"],right_on=["Account","Type"],validate="many_to_one",copy=False)
df_transform.rename(columns={"Time_x":"Time","Time_y":"Time_median"},inplace=True)

In [ ]:
df_transform = df_transform.merge(df_transform.groupby(by=['Account'])['Volume'].sum().reset_index(),how="inner",left_on=["Account"],right_on=["Account"],validate="many_to_one",copy=False)
df_transform.rename(columns={"Volume_x":"Volume","Volume_y":"Sum_of_vol"},inplace=True)

In [ ]:
df_transform = df_transform.merge(df_transform.groupby(by=['Account',"Type","Symbol"])['Volume'].sum().reset_index(),how="inner",left_on=['Account',"Type","Symbol"],right_on=['Account',"Type","Symbol"],validate="many_to_one",copy=False)
df_transform.rename(columns={"Volume_x":"Volume","Volume_y":"Sum_of_vol_type"},inplace=True)

In [ ]:
df_transform = df_transform.merge(df_transform.groupby(by=['Account'])['Symbol'].unique().map(lambda x: len(x)).reset_index(),how="inner",left_on=['Account'],right_on=['Account'],validate="many_to_one",copy=False)
df_transform.rename(columns={"Symbol_x":"Symbol","Symbol_y":"Num_of_sym"},inplace=True)

In [ ]:
df_transform = df_transform[df_transform.Reason.map(lambda x: x in ['Client','Mobile'])]

In [ ]:
df_transform.reset_index(drop=True,inplace=True)

In [ ]:
df_group = df_transform.copy()

**Đầu tiên ta sẽ group mỗi account thành 2 records với mỗi record chứa dữ liệu sell tất cả các loại tiền, buy tất cả các loại tiền**

In [ ]:
data_buy = df_group[df_group.Type == "buy"]
data_sell = df_group[df_group.Type == "sell"]

In [ ]:
sum_data_buy = data_buy.groupby(by=['Account',"Symbol"])['Volume'].sum().reset_index()
data_buy = data_buy.groupby(by=['Account']).head(1)
data_buy.reset_index(drop=True,inplace=True)
sum_data_buy = sum_data_buy.pivot(index='Account', columns='Symbol', values='Volume').fillna(0).reset_index()
data_buy = data_buy.merge(sum_data_buy,how='inner',on="Account")

In [ ]:
sum_data_sell = data_sell.groupby(by=['Account',"Symbol"])['Volume'].sum().reset_index()
data_sell = data_sell.groupby(by=['Account']).head(1)
data_sell.reset_index(drop=True,inplace=True)
sum_data_sell = sum_data_sell.pivot(index='Account', columns='Symbol', values='Volume').fillna(0).reset_index()
data_sell = data_sell.merge(sum_data_sell,how='inner',on="Account")

In [ ]:
df_group = pd.concat([data_sell,data_buy]).fillna(0)

**Bởi vì profit của cả 2 account sẽ ngược nhau nên ta sẽ lấy trị tuyệt đối của profit để so sánh**

In [ ]:
df_group.reset_index(drop=True,inplace=True)

In [ ]:
df_group[df_group.Account==200147580]

,Account,Group,Reason,Time,Type,Symbol,Volume,Price,Swap,Balance,Bonus,Reg.date,Profit,Comment,Num_of_order,Time_median,Sum_of_vol,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym
4486,200147580,GF-B-JPY-GM2,Mobile,383134.675804,sell,usdjpy,1.0,104.883,144.0,311856.0,600025.0,2020.01.20 15:47,79100.0,0,8,383158.175804,8.0,8.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_group[df_group.Account==200121490]

,Account,Group,Reason,Time,Type,Symbol,Volume,Price,Swap,Balance,Bonus,Reg.date,Profit,Comment,Num_of_order,Time_median,Sum_of_vol,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym
10731,200121490,GF-B-JPY-GM2,Mobile,383247.675804,buy,usdjpy,4.0,104.897,426.0,300000.0,600947.0,2019.10.30 15:30,-328000.0,0,2,383249.175804,8.0,8.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Sử dụng min max scaler với các cột chứa tổng các loại tiền**

In [ ]:
scaler_column = df_group.drop(columns=['Sum_of_vol','Num_of_order','Account', 'Group', 'Reason', 'Time', 'Type', 'Symbol', 'Volume',
       'Price', 'Swap','Reg.date','Comment',"Profit"]).columns

In [ ]:
scaler_column

Index(['Balance', 'Bonus', 'Time_median', 'Sum_of_vol_type', 'Num_of_sym',
       'audcad', 'audchf', 'audjpy', 'audnzd', 'audusd', 'audusdm', 'caceur',
       'cadjpy', 'chfjpy', 'daxeur', 'dowusd', 'esxeur', 'euraud', 'eurcad',
       'eurchf', 'eurgbp', 'eurjpy', 'eurnzd', 'eurnzd.cn', 'eurusd',
       'eurusd.cn', 'eurusdm', 'eurzar', 'ftxgbp', 'gbpaud', 'gbpcad',
       'gbpchf', 'gbpjpy', 'gbpnzd', 'gbpusd', 'gbpusdm', 'hsxhkd', 'nasusd',
       'nzdjpy', 'nzdusd', 's&pusd', 'usdcad', 'usdchf', 'usdcnh', 'usdjpy',
       'usdmxn', 'usdsgd', 'usdtry', 'usdzar', 'wtiusd', 'xagusd', 'xauusd',
       'usdchfm', 'usdjpym'],
      dtype='object')

In [ ]:
scaler_minmax,df_scaler = mt.transform(df_group,scaler_column,scaler='minmax')

In [ ]:
df_scaler['Account'] = df_group.Account
df_scaler['Num_of_order'] = df_group.Num_of_order
df_scaler['Type'] = df_group.Type
df_scaler['Sum_of_vol'] = df_group.Sum_of_vol

**Chia dữ liệu đã scale lại thành 2 dataframe buy và sell, sau đó là tìm những account đối xứng của data buy từ data sell và ngược lại**

In [ ]:
data_buy = df_scaler[df_scaler.Type == "buy"]
data_sell = df_scaler[df_scaler.Type == "sell"]
data_buy.reset_index(drop=True,inplace=True)
data_sell.reset_index(drop=True,inplace=True)

**Sử dụng NearestNeightbors của sklearn với n_neighbors=1 và fit với data_buy sau đó là tìm những account gần với data_sell trong data_buy**

In [ ]:
data_buy.drop(columns=['Account','Type']).columns

Index(['Balance', 'Bonus', 'Time_median', 'Sum_of_vol_type', 'Num_of_sym',
       'audcad', 'audchf', 'audjpy', 'audnzd', 'audusd', 'audusdm', 'caceur',
       'cadjpy', 'chfjpy', 'daxeur', 'dowusd', 'esxeur', 'euraud', 'eurcad',
       'eurchf', 'eurgbp', 'eurjpy', 'eurnzd', 'eurnzd.cn', 'eurusd',
       'eurusd.cn', 'eurusdm', 'eurzar', 'ftxgbp', 'gbpaud', 'gbpcad',
       'gbpchf', 'gbpjpy', 'gbpnzd', 'gbpusd', 'gbpusdm', 'hsxhkd', 'nasusd',
       'nzdjpy', 'nzdusd', 's&pusd', 'usdcad', 'usdchf', 'usdcnh', 'usdjpy',
       'usdmxn', 'usdsgd', 'usdtry', 'usdzar', 'wtiusd', 'xagusd', 'xauusd',
       'usdchfm', 'usdjpym', 'Num_of_order', 'Sum_of_vol'],
      dtype='object')

In [ ]:
data_sell.drop(columns=['Account','Type']).columns

Index(['Balance', 'Bonus', 'Time_median', 'Sum_of_vol_type', 'Num_of_sym',
       'audcad', 'audchf', 'audjpy', 'audnzd', 'audusd', 'audusdm', 'caceur',
       'cadjpy', 'chfjpy', 'daxeur', 'dowusd', 'esxeur', 'euraud', 'eurcad',
       'eurchf', 'eurgbp', 'eurjpy', 'eurnzd', 'eurnzd.cn', 'eurusd',
       'eurusd.cn', 'eurusdm', 'eurzar', 'ftxgbp', 'gbpaud', 'gbpcad',
       'gbpchf', 'gbpjpy', 'gbpnzd', 'gbpusd', 'gbpusdm', 'hsxhkd', 'nasusd',
       'nzdjpy', 'nzdusd', 's&pusd', 'usdcad', 'usdchf', 'usdcnh', 'usdjpy',
       'usdmxn', 'usdsgd', 'usdtry', 'usdzar', 'wtiusd', 'xagusd', 'xauusd',
       'usdchfm', 'usdjpym', 'Num_of_order', 'Sum_of_vol'],
      dtype='object')

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2,p=2,algorithm="kd_tree")
nbrs.fit(data_buy.drop(columns=['Account','Type','Num_of_order','Sum_of_vol','Num_of_sym','Sum_of_vol_type']))

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                 radius=1.0)

In [ ]:
distances, indices = nbrs.kneighbors(data_sell.drop(columns=['Account','Type','Num_of_order','Sum_of_vol','Num_of_sym','Sum_of_vol_type']))

In [ ]:
indices_df = pd.DataFrame(indices,columns=['Account_0','Account_1'])
distances_df = pd.DataFrame(distances,columns=['Distance_0','Distance_1'])

In [ ]:
data_sell['Account_1'] = data_buy.iloc[indices_df.Account_0].Account.values
data_sell['Account_2'] = data_buy.iloc[indices_df.Account_1].Account.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_sell['distance'] = distances_df.Distance_0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


**Do có khá nhiều account account giống nhau nhưng trade rất nhỏ nên distance cũng rất nhỏ vì vậy phải filter lại với một số điều kiện để lọc được những đối tượng khả nghi**

In [ ]:
data_buy[data_buy.Account==200210077]

,Balance,Bonus,Time_median,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym,Account,Num_of_order,Type,Sum_of_vol
6062,0.068166,0.019956,0.060425,0.0114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200210077,8,buy,8.0


In [ ]:
data_buy[data_buy.Account==5345903]

,Balance,Bonus,Time_median,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym,Account,Num_of_order,Type,Sum_of_vol
1375,0.063162,0.022384,0.000364,0.034258,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5345903,18,buy,24.0


In [ ]:
data_buy[data_buy.Account==200121490]

,Balance,Bonus,Time_median,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym,Account,Num_of_order,Type,Sum_of_vol
5188,0.070117,0.066624,0.002493,0.022829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200121490,2,buy,8.0


In [ ]:
data_sell[data_sell.Account==200147580]

,Balance,Bonus,Time_median,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym,Account,Num_of_order,Type,Sum_of_vol,Account_1,Account_2,distance
4486,0.07053,0.066522,0.002492,0.022829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200147580,8,sell,8.0,200121490,5371893,0.000425


In [ ]:
pd.set_option("display.max_rows",100)
data_sell[(data_sell.Sum_of_vol > 6) & (data_sell.Account != data_sell.Account_1) & (data_sell.Num_of_order <=8)].sort_values(by="distance",ascending=True)[["Account","Sum_of_vol","Account_1",'Account_2',"distance",'Num_of_order']].head(50)

,Account,Sum_of_vol,Account_1,Account_2,distance,Num_of_order
1860,5367447,20.00,5365883,5375380,9.583199e-09,8
1765,5363061,25.00,5369097,200106233,1.916640e-07,1
1856,5358921,10.00,5371826,5369521,2.069971e-06,3
1622,5349295,25.00,200143013,5373143,2.258659e-04,5
4486,200147580,8.00,200121490,5371893,4.247823e-04,8
1678,5364043,19.00,5369959,200106233,1.315944e-03,2
4313,200052832,10.00,5167682,5349961,1.922610e-03,2
3492,200130878,8.90,200143223,5364589,2.199860e-03,7
4748,200182811,7.50,200164872,200203868,2.384301e-03,8
3716,200217118,10.00,5349295,200036078,2.474536e-03,1


In [ ]:
data_buy.head(2)

,Balance,Bonus,Time_median,Sum_of_vol_type,Num_of_sym,audcad,audchf,audjpy,audnzd,audusd,audusdm,caceur,cadjpy,chfjpy,daxeur,dowusd,esxeur,euraud,eurcad,eurchf,eurgbp,eurjpy,eurnzd,eurnzd.cn,eurusd,eurusd.cn,eurusdm,eurzar,ftxgbp,gbpaud,gbpcad,gbpchf,gbpjpy,gbpnzd,gbpusd,gbpusdm,hsxhkd,nasusd,nzdjpy,nzdusd,s&pusd,usdcad,usdchf,usdcnh,usdjpy,usdmxn,usdsgd,usdtry,usdzar,wtiusd,xagusd,xauusd,usdchfm,usdjpym,Account,Num_of_order,Type,Sum_of_vol
0,0.067664,0.018783,0.000001,0.003400,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060,0.0,0.0,5346027,2,buy,1.2
1,0.066624,0.022193,0.000002,0.000257,0.0625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005,0.0,0.0,5351989,7,buy,0.6


________________________________________________________